In [1]:
from pset_5.task import TargetOutput, Requires, Requirement
from pset_5.dask.target import CSVTarget, ParquetTarget

from luigi import ExternalTask, Parameter, Task, format, LocalTarget, DateParameter, BoolParameter
from luigi.contrib.s3 import S3Target
from dask.dataframe import read_csv, read_parquet, to_csv, to_parquet
import pandas as pd




dsk = read_parquet("data/fullset/*.parquet")
print(dsk.dtypes)
print(dsk.head())

review_id                   object
user_id                     object
business_id                 object
stars                        int32
date           datetime64[ns, UTC]
text                        object
useful                       int32
funny                        int32
cool                         int32
dtype: object
                    review_id                 user_id             business_id  \
index                                                                           
0      o1qYw1U8wGghaetGVFpvDQ  pJxbP5EuMgHI-2VtibAgmA  mJFdBXviC-1YRtY51XnS6A   
1      yhEjeiKaIy8TWJL42Xgp1w  IR1DdN4wwkc4fI0Qv5KyYg  e4NQLZynhSmvwl38hC4m-A   
2      1US3ayDP-yVI5sXyx5Sk-g  ZQBZq-RZqObP0xchzX7p8w  F1odkZWmcXGylij8HjRq6Q   
3      MisNiCS065C5GqJ-IYWuvQ  2E9e6OQ7Cb0E_xtHHyEX7A  plEghR2-XhS5WtZAmp-x5Q   
4      FKXILR2RkYPejJ2shXlx7g  sjxGOtb7E-qnuLHEMuFX2w  Bx7SDDMr5Y278PFIy0nsmQ   

       stars       date                                               text  \
index                    

In [222]:
dsk['review_id'].str.len

functools.partial(<bound method Accessor._function_map of <dask.dataframe.accessor.StringAccessor object at 0x0000021066C83DD8>>, 'len')

In [223]:
type(dsk)

dask.dataframe.core.DataFrame

In [224]:
len(dsk)

199943

In [225]:
len(dsk.columns)

9

In [226]:
dsk=dsk[~(~(dsk.review_id.str.len() == 22) | dsk['user_id'].isnull())]

In [227]:
len(dsk)

199943

In [228]:
pd.isnull(dsk['useful']).sum() 

0

In [229]:
dsk[['useful','funny', 'stars', 'cool']]= dsk[['useful','funny','stars','cool']].fillna(0)

In [230]:
pd.isnull(dsk['useful']).sum() 

0

In [231]:
pd.isnull(dsk['cool']).sum() 

0

In [232]:
dsk.dtypes

review_id                   object
user_id                     object
business_id                 object
stars                        int32
date           datetime64[ns, UTC]
text                        object
useful                       int32
funny                        int32
cool                         int32
dtype: object

In [175]:
dsk=dsk[~(~(dsk.review_id.str.len() == 22) | dsk['user_id'].isnull())]
print(len(dsk))
dsk[['useful','funny', 'stars', 'cool']]= dsk[['useful','funny','stars','cool']].astype(int)
dsk.dtypes

199943


review_id                   object
user_id                     object
business_id                 object
stars                        int32
date           datetime64[ns, UTC]
text                        object
useful                       int32
funny                        int32
cool                         int32
dtype: object

In [176]:
dsk1 = dsk
type(dsk1)
dsk['text_length']=dsk['text'].str.len()
dsk.text_length.head()

index
0     182
1     336
2     651
3    1854
4     607
Name: text_length, dtype: int64

In [177]:
dsk.groupby('stars').text_length.mean().compute()

stars
1    755.244306
2    762.685914
3    709.630383
4    622.908382
5    485.507673
Name: text_length, dtype: float64

In [234]:
dsk.date.dt.year.min().compute()

2005.0

In [246]:
dsk = read_parquet("data/fullset/*.parquet")
#print(dsk.dtypes)
#print(dsk.head())


dsk = dsk[~(~(dsk.review_id.str.len() == 22) | dsk['user_id'].isnull())]
dsk[['useful', 'funny', 'stars', 'cool']] = dsk[['useful', 'funny', 'stars', 'cool']].fillna(0)
dsk[['useful', 'funny', 'stars', 'cool']] = dsk[['useful', 'funny', 'stars', 'cool']].astype(int)



dsk['decade'] = (dsk.date.dt.year.mod(100)/10)
dsk['decade'].isnull().sum().compute()
dsk = dsk[~dsk['decade'].isnull()]
dsk['decade'].isnull().sum().compute()
dsk['decade'] = dsk['decade'].astype(int)
dsk['text_length']=dsk['text'].str.len()
dsk.groupby('decade').text_length.mean().astype(int).compute()

#dsk=dsk[~dsk['decade'].isnull()]
#dsk['decade'].isnull().compute()

decade
0    776
1    598
Name: text_length, dtype: int32

AttributeError: 'Column not found: text_length'